In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN


In [17]:
mx_localities=pd.read_csv('https://raw.githubusercontent.com/spudnik99/K-means_for_Geospatial_data_using_Haversine/master/AGEEML_20212101236315.csv',dtype={'CVE_LOC_ALL': str, 'CVE_ENT_TXT': str,'CVE_MUN_ALL': str, 'CVE_LOC_TXT': str,'Lat_Decimal':float,'Lon_Decimal':float,'Pob_Total':int})

c:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning:

Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.



In [18]:
mx_localities_out=mx_localities[mx_localities["Pob_Total"]>2000]
coords=mx_localities_out[['Lat_Decimal','Lon_Decimal']].to_numpy()
kms_per_radian = 6371.0088
epsilon = 50.5 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))
print('Clusters: {}'.format(clusters))

Number of clusters: 67
Clusters: 0     [[21.879823, -102.296047], [22.016097, -102.14...
1     [[31.808944, -116.595134], [32.096985, -116.57...
2     [[32.641176, -115.475579], [32.228011, -115.17...
3                             [[31.02292, -114.838901]]
4     [[25.034636, -111.673339], [25.192906, -112.11...
                            ...                        
62                            [[25.263505, -97.432246]]
63    [[27.486291, -99.508493], [27.473336, -99.6217...
64    [[24.848678, -98.159621], [25.017079, -98.07896]]
65                             [[24.62555, -97.717085]]
66                           [[24.613169, -101.420484]]
Length: 67, dtype: object


In [19]:
import plotly.graph_objects as go

fig = go.Figure()

#Loop through clusters to add to map
for i in range(num_clusters):
    df_clusters=pd.DataFrame(clusters[i],columns=["Lat_Decimal","Lon_Decimal"])
    fig.add_trace(go.Scattermapbox(lat=df_clusters.Lat_Decimal, lon=df_clusters.Lon_Decimal,mode='markers',
        marker=go.scattermapbox.Marker(
            size=17,
            color=10+(2*i),#'rgb(255, 0, 0)',
            opacity=0.7
        ), text='Cluster:'+str(i)))



fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(autosize=True,hovermode='closest',pitch=30,zoom=10)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()